In [1]:
import psycopg2, time

In [ ]:
state_codes = {
    'al': '01', 'ak': '02', 'az': '04', 'ar': '05', 'ca': '06',
    'co': '08', 'ct': '09', 'de': '10', 'dc': '11', 'fl': '12',
    'ga': '13', 'hi': '15', 'id': '16', 'il': '17', 'in': '18',
    'ia': '19', 'ks': '20', 'ky': '21', 'la': '22', 'me': '23',
    'md': '24', 'ma': '25', 'mi': '26', 'mn': '27', 'ms': '28',
    'mo': '29', 'mt': '30', 'ne': '31', 'nv': '32', 'nh': '33',
    'nj': '34', 'nm': '35', 'ny': '36', 'nc': '37', 'nd': '38',
    'oh': '39', 'ok': '40', 'or': '41', 'pa': '42', 'ri': '44',
    'sc': '45', 'sd': '46', 'tn': '47', 'tx': '48', 'ut': '49',
    'vt': '50', 'va': '51', 'wa': '53', 'wi': '54', 'wy': '55',
    'wv': '56'
}

state_ids = sorted(state_codes.values())
state_names = sorted(state_codes.keys())

In [11]:
def psql(cmds, database='timelapse'):
    cmd = ['psql', '-d', database]
    before = time.time()
    p = subprocess.Popen(cmd,
                         stdin=subprocess.PIPE,
                         stdout=subprocess.PIPE,
                         stderr=subprocess.PIPE)
    # Send cmds (string) to psql while collecting stdout, stderr
    sys.stdout.write('%s\n' % cmds.strip())
    (out, err) = p.communicate(cmds)
    p.wait()
    sys.stdout.write('Finished execution in %g secs: %s\n' % (time.time() - before, (out + '\n' + err).strip())) 
    
    
def query_psql(query, quiet=False):
    conn = psycopg2.connect(dbname='timelapse', host='/var/run/postgresql')
    before = time.time()
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    cur.close()
    elapsed = time.time() - before
    if not quiet:
        sys.stdout.write('Execution of %s\ntook %g seconds and returned %d rows\n' % (query.strip(), elapsed, len(rows)))
    return rows

In [3]:
def download_file(url, filename):
    if os.path.exists(filename):
        sys.stdout.write('%s already downloaded\n' % filename)
    else:
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        sys.stdout.write('Downloading %s to %s\n' % (url, filename))
        data = urllib2.urlopen(url).read()
        open(filename + '.tmp', "wb").write(data)
        os.rename(filename + '.tmp', filename)
        sys.stdout.write('Done, wrote %d bytes to %s\n' % (len(data), filename))

def unzip_file(filename):
    exdir = os.path.splitext(filename)[0]
    if os.path.exists(exdir):
        sys.stdout.write('%s already unzipped\n' % (filename))
    else:
        tmpdir = exdir + '.tmp'
        shutil.rmtree(tmpdir, True)
        sys.stdout.write('Unzipping %s into %s\n' % (filename, tmpdir))
        subprocess.check_call(['unzip', filename, '-d', tmpdir])
        os.rename(tmpdir, exdir)
        print 'Success, created %s' % (exdir)
        
def gunzip_file(filename):
    dest = os.path.splitext(filename)[0]
    if os.path.exists(dest):
        sys.stdout.write('%s already unzipped\n' % (filename))
    else:
        tmp = dest + '.tmp'
        sys.stdout.write('gunzipping %s\n' % (filename))
        subprocess.check_call("gunzip -c '%s' > '%s'" % (filename, tmp), shell=True)
        os.rename(tmp, dest)
        sys.stdout.write('Success, created %s\n' % (dest))